## Custom Model

### Tensorflow (-Slim)
First, define your model (its endpoints) and its preprocessing function.

In [1]:
from model_tools.activations.tensorflow import load_resize_image
import tensorflow as tf
slim = tf.contrib.slim
tf.reset_default_graph()

image_size = 224
placeholder = tf.placeholder(dtype=tf.string, shape=[64])
preprocess = lambda image_path: load_resize_image(image_path, image_size)
preprocess = tf.map_fn(preprocess, placeholder, dtype=tf.float32)

with tf.variable_scope('my_model', values=[preprocess]) as sc:
    end_points_collection = sc.original_name_scope + '_end_points'
    # Collect outputs for conv2d, fully_connected and max_pool2d.
    with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                        outputs_collections=[end_points_collection]):
        net = slim.conv2d(preprocess, 64, [11, 11], 4, padding='VALID', scope='conv1')
        net = slim.max_pool2d(net, [5, 5], 5, scope='pool1')
        net = slim.max_pool2d(net, [3, 3], 2, scope='pool2')
        net = slim.flatten(net, scope='flatten')
        net = slim.fully_connected(net, 1000, scope='logits')
        endpoints = slim.utils.convert_collection_to_dict(end_points_collection)

session = tf.Session()
session.run(tf.initialize_all_variables())

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Then, using the TensorflowSlimWrapper, convert your model into an activations model.

In [2]:
from model_tools.activations.tensorflow import TensorflowSlimWrapper

activations_model = TensorflowSlimWrapper(identifier='tf-custom', labels_offset=0,
                                          endpoints=endpoints, inputs=placeholder, session=session)

Model candidates in Brain-Score have to follow the [model_interface](https://github.com/brain-score/brain-score/blob/master/brainscore/model_interface.py).
The main step for most ML models to map onto the brain model interface is deciding what layers map onto cortical regions.
If you know which layer corresponds to which region beforehand, you can just use a `LayerMappedModel` and `commit` a layer to a region.

In [3]:
from brainscore import score_model
from model_tools.brain_transformation import LayerMappedModel, TemporalIgnore

# layer -> region
model = LayerMappedModel("tf-custom-pool2", activations_model=activations_model, visual_degrees=8)
model.commit("IT", "my_model/pool2")
# ignore time_bins
model = TemporalIgnore(model)

score = score_model(model_identifier=model.identifier, model=model,
                    benchmark_identifier='dicarlo.Majaj2015public.IT-pls')
print(score)

cross-validation: 100%|██████████| 10/10 [00:20<00:00,  2.04s/it]


<xarray.Score (aggregation: 2)>
array([0.285252, 0.004027])
Coordinates:
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:                   <xarray.Score (aggregation: 2)>\narray([0.43571 , ...
    ceiling:               <xarray.Score (aggregation: 2)>\narray([0.815799, ...
    model_identifier:      tf-custom-pool2
    benchmark_identifier:  dicarlo.Majaj2015public.IT-pls


You can also empirically choose the best layer for a particular region. You might want to use PCA for large layers.

In [4]:
from model_tools.brain_transformation import LayerSelection
from model_tools.activations.pca import LayerPCA
from brainscore.benchmarks.public_benchmarks import MajajITPublicBenchmark

# select best layer based on separate data
benchmark = MajajITPublicBenchmark()
layer_selection = LayerSelection(model_identifier=model.identifier, activations_model=activations_model,
                                 layers=['my_model/conv1', 'my_model/pool1', 'my_model/pool2'], visual_degrees=8)
hook = LayerPCA.hook(activations_model, n_components=1000)
best_layer = layer_selection(selection_identifier='IT', benchmark=benchmark)
hook.remove()
print(f'Using layer {best_layer} for IT')

# commit best layer
model = LayerMappedModel(model.identifier, activations_model=activations_model, visual_degrees=8)
model.commit('IT', best_layer)

# benchmarks also give us information about which time bins are being recorded.
# since this is a feed-forward model with no timesteps, we ignore that and always yield the same representations
model = TemporalIgnore(model)

# evaluate committed model
score = score_model(model_identifier=model.identifier, model=model,
                    benchmark_identifier='dicarlo.Majaj2015public.IT-pls')
print(score)

Using layer my_model/pool2 for IT
<xarray.Score (aggregation: 2)>
array([0.285252, 0.004027])
Coordinates:
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:                   <xarray.Score (aggregation: 2)>\narray([0.43571 , ...
    ceiling:               <xarray.Score (aggregation: 2)>\narray([0.815799, ...
    model_identifier:      tf-custom-pool2
    benchmark_identifier:  dicarlo.Majaj2015public.IT-pls


cross-validation: 100%|██████████| 10/10 [00:21<00:00,  2.12s/it]


All of these steps are sample implementations and as long as you implement the model_interface,
it does not matter how you get there.

## Pre-defined models

Scoring a model on neural data can be done in a single line using the `score_model` method and the `brain_translated_pool`.
Pre-defined layers of a model will be used to retrieve the activations.
Just like with the model implementations, the result of this method call will be cached 
so that it only needs to be computed once.


In [5]:
from brainscore import score_model
from candidate_models.model_commitments import brain_translated_pool

identifier = 'alexnet'
model = brain_translated_pool[identifier]
score = score_model(model_identifier=identifier, model=model, benchmark_identifier='dicarlo.Majaj2015public.IT-pls')
print(score)



<xarray.Score (aggregation: 2)>
array([0.50479 , 0.002694])
Coordinates:
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:                   <xarray.Score (aggregation: 2)>\narray([0.579614, ...
    ceiling:               <xarray.Score (aggregation: 2)>\narray([0.815799, ...
    model_identifier:      alexnet
    benchmark_identifier:  dicarlo.Majaj2015public.IT-pls


alexnet is accessed again and reloaded
cross-validation: 100%|██████████| 10/10 [02:02<00:00, 12.25s/it]


A score typically comes with an estimate of the center (e.g. mean) and error (e.g. standard error of the mean).
These values are aggregations over splits and often neuroids, and ceiled by the benchmark ceiling.

Check out https://github.com/brain-score/brain-score/blob/master/examples/benchmarks.ipynb for more details.

Also note that all these scores were computed on the publicly available data.
To test models on the full set of benchmarks (including held-out private data), 
please submit them on www.Brain-Score.org.